<a href="https://colab.research.google.com/github/MehrdadJalali-AI/MOF_LENS/blob/main/MOF_LENS_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')
# Change working path
os.chdir('/content/drive/MyDrive/Research/MOF/MOF-LENS/')
!pip install rdkit
!pip install deap

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import gamma
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from joblib import Parallel, delayed
import time
import os
from rdkit import Chem, RDLogger
from rdkit.Chem import rdFingerprintGenerator, DataStructs, Draw

# Suppress RDKit warnings and errors to reduce verbosity
RDLogger.DisableLog('rdApp.*')
RDLogger.DisableLog('rdApp.error')  # Suppress valence error logs (temporary)

# Load and preprocess dataset
file_path = "MOF.csv"
df = pd.read_csv(file_path)
expected_features = ['Refcode', 'void_fraction', 'asa (A^2)', 'pld (A)', 'max_metal_coordination_n',
                    'n_sbu_point_of_extension']
smiles_columns = ['ligand_smile', 'linker_smile', 'metal_sbu_smile', 'metal_cluster_smile']  # Prioritize ligand_smile
available_smiles_columns = [col for col in smiles_columns if col in df.columns]
if 'metals' in df.columns:
    expected_features.append('metals')
available_features = [col for col in expected_features if col in df.columns]
if not all(f in df.columns for f in expected_features):
    raise KeyError(f"Missing required columns: {[f for f in expected_features if f not in df.columns]}")
df = df[available_features + available_smiles_columns]
numerical_features = [f for f in available_features if f not in ['Refcode', 'metals']]

# Select SMILES column (priority: ligand_smile > linker_smile > others)
selected_smiles_column = None
for col in ['ligand_smile', 'linker_smile', 'metal_sbu_smile', 'metal_cluster_smile']:
    if col in df.columns:
        selected_smiles_column = col
        break
if selected_smiles_column:
    print(f"Using SMILES column: {selected_smiles_column}")
    print(f"Non-null SMILES count: {df[selected_smiles_column].notna().sum()}")
    print(f"Sample SMILES: {df[selected_smiles_column].dropna().head().tolist()}")
else:
    print("No SMILES columns found. Using default fingerprints (methane).")

# Filter toxic metals
if 'metals' in df.columns:
    toxic_metals = ['Pb', 'Cd', 'Cr', 'Ni', 'Hg']
    df = df[~df['metals'].str.contains('|'.join(toxic_metals), case=False, na=False)]

# Filter valid SMILES
if selected_smiles_column:
    def is_valid_smiles(smiles):
        if not smiles or pd.isna(smiles):
            return False
        try:
            mol = Chem.MolFromSmiles(smiles, sanitize=True)
            return mol is not None
        except:
            return False
    valid_smiles_mask = df[selected_smiles_column].apply(is_valid_smiles)
    invalid_smiles = df[~valid_smiles_mask][['Refcode', selected_smiles_column]].dropna()
    if not invalid_smiles.empty:
        print(f"Found {len(invalid_smiles)} invalid SMILES:")
        for _, row in invalid_smiles.head(10).iterrows():
            print(f"Invalid SMILES for Refcode {row['Refcode']}: {row[selected_smiles_column]}")
    df = df[valid_smiles_mask]
    print(f"Valid SMILES count: {len(df)}")
    if len(df) == 0:
        print("No valid SMILES found. Using default fingerprints (methane).")
        selected_smiles_column = None

# Subsample dataset if too large
if len(df) > 10000:
    df = df.sample(n=10000, random_state=42)
    print(f"Subsampled dataset to {len(df)} MOFs")

# Compute Morgan fingerprints
def get_morgan_fingerprint(smiles, radius=2, nBits=512, default_mol=None, refcode=None):
    if default_mol is None:
        default_mol = Chem.MolFromSmiles('C')
        default_fp_gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
        default_fp = default_fp_gen.GetFingerprint(default_mol)
        default_arr = np.zeros(nBits, dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(default_fp, default_arr)
        default_result = (default_arr, default_fp)
    else:
        default_result = default_mol
    if not smiles or pd.isna(smiles):
        print(f"Invalid SMILES for Refcode {refcode}: {smiles}, using default (methane)")
        return default_result
    try:
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
        if mol is None:
            print(f"Invalid SMILES for Refcode {refcode}: {smiles}, using default (methane)")
            return default_result
        fp_gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
        fp = fp_gen.GetFingerprint(mol)
        arr = np.zeros(nBits, dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(fp, arr)
        return arr, fp
    except Exception as e:
        print(f"Error processing SMILES for Refcode {refcode}: {smiles}, error: {e}, using default (methane)")
        return default_result

default_mol = Chem.MolFromSmiles('C')
default_fp_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=512)
default_fp = default_fp_gen.GetFingerprint(default_mol)
default_arr = np.zeros(512, dtype=np.uint8)
DataStructs.ConvertToNumpyArray(default_fp, default_arr)
default_result = (default_arr, default_fp)

if selected_smiles_column:
    fingerprint_results = Parallel(n_jobs=-1)(
        delayed(get_morgan_fingerprint)(smiles, default_mol=default_result, refcode=refcode)
        for smiles, refcode in zip(df[selected_smiles_column], df['Refcode'])
    )
    df['Fingerprint'] = [res[0] for res in fingerprint_results]
    df['RDKit_Fingerprint'] = [res[1] for res in fingerprint_results]
else:
    df['Fingerprint'] = [default_result[0] for _ in range(len(df))]
    df['RDKit_Fingerprint'] = [default_result[1] for _ in range(len(df))]

# Reference molecule (ibuprofen)
reference_smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
reference_fp, reference_fp_rdkit = get_morgan_fingerprint(reference_smiles, refcode="Ibuprofen")

# Normalize features
df_norm = df.copy()
min_vals = df[numerical_features].min()
max_vals = df[numerical_features].max()
df_norm[numerical_features] = (df[numerical_features] - min_vals) / (max_vals - min_vals)

# Define ranges (relaxed)
raw_ranges = {feat: (max(df[feat].quantile(0.25) - 1.0 * (df[feat].quantile(0.75) - df[feat].quantile(0.25)), df[feat].min()),
                     min(df[feat].quantile(0.75) + 1.0 * (df[feat].quantile(0.75) - df[feat].quantile(0.25)), df[feat].max()))
              for feat in numerical_features}
ranges = {feat: ((raw_ranges[feat][0] - min_vals[feat]) / (max_vals[feat] - min_vals[feat]),
                 (raw_ranges[feat][1] - min_vals[feat]) / (max_vals[feat] - min_vals[feat]))
          for feat in numerical_features}

# Pre-filter dataset
mask = False
for feat, (min_val, max_val) in raw_ranges.items():
    mask |= df[feat].between(min_val, max_val)
df = df[mask]
df_norm = df_norm[mask]

# Prepare kNN
numerical_data = df_norm[numerical_features].values
fingerprint_data = np.stack(df_norm['Fingerprint'].values)
combined_data = np.hstack([numerical_data, fingerprint_data])
numerical_indices = list(range(len(numerical_features)))
fp_index = len(numerical_features)

# Precompute Tanimoto matrix
def tanimoto_similarity(fp1, fp2):
    fp1 = fp1.astype(np.uint8)
    fp2 = fp2.astype(np.uint8)
    if np.sum(fp1) == 0 and np.sum(fp2) == 0:
        return 0.0
    intersection = np.sum(fp1 & fp2)
    union = np.sum(fp1 | fp2)
    return intersection / union if union > 0 else 0.0

def precompute_tanimoto_matrix(fingerprints):
    n = len(fingerprints)
    tanimoto_matrix = np.zeros((n, n))
    def compute_pair(i, j):
        return i, j, tanimoto_similarity(fingerprints[i], fingerprints[j])
    pairs = [(i, j) for i in range(n) for j in range(i + 1, n)]
    results = Parallel(n_jobs=-1)(delayed(compute_pair)(i, j) for i, j in pairs)
    for i, j, sim in results:
        tanimoto_matrix[i, j] = sim
        tanimoto_matrix[j, i] = sim
    return tanimoto_matrix

tanimoto_matrix = precompute_tanimoto_matrix(fingerprint_data)

def custom_metric(x, y, numerical_indices, fp_index, tanimoto_matrix, indices):
    num_dist = np.sqrt(np.sum((x[numerical_indices] - y[numerical_indices])**2))
    i, j = indices[0], indices[1]
    tanimoto = tanimoto_matrix[i, j]
    chem_dist = 1 - tanimoto
    return 0.7 * num_dist + 0.3 * chem_dist

nbrs = NearestNeighbors(n_neighbors=1, metric=lambda x, y: custom_metric(
    x, y, numerical_indices, fp_index, tanimoto_matrix, [int(x[-1]), int(y[-1])]
), algorithm='brute').fit(np.hstack([combined_data, np.arange(len(combined_data)).reshape(-1, 1)]))

# Define toxicity estimation
def estimate_toxicity(metal=None):
    if metal is None:
        return 0.0
    toxicity_scores = {'Zn': 0.2, 'Cu': 0.3, 'Fe': 0.1}
    return toxicity_scores.get(metal, 0.5)

# Fitness function (batch)
def fitness_function_drug_delivery_batch(solutions, df_norm, ranges, nbrs, df, top_solutions, reference_fp_rdkit):
    solution_nums = solutions[:, :len(numerical_features)]
    solution_fp = df_norm.iloc[0]['Fingerprint']
    solution_combined = np.hstack([solution_nums, np.tile(solution_fp, (len(solutions), 1)),
                                  np.arange(len(solutions)).reshape(-1, 1)])
    mof_indices = nbrs.kneighbors(solution_combined, n_neighbors=1, return_distance=False)[:, 0]
    results = []
    for idx, solution_num in zip(mof_indices, solution_nums):
        mapped_solution = df_norm[numerical_features].iloc[idx].values
        void_fraction, asa, pld, metal_coord, sbu_points = mapped_solution
        toxicity = estimate_toxicity(df.iloc[idx]['metals'] if 'metals' in df.columns else None)
        mof_fp_rdkit = df_norm.iloc[idx]['RDKit_Fingerprint']
        weights = {
            'void_fraction': 0.25, 'asa': 0.25, 'pld': 0.3, 'metal_coord': 0.1,
            'sbu_points': 0.1, 'toxicity': 0.05, 'chemical': 0.35
        }
        structural_score = (weights['void_fraction'] * void_fraction +
                           weights['asa'] * asa +
                           weights['pld'] * pld +
                           weights['metal_coord'] * metal_coord +
                           weights['sbu_points'] * sbu_points -
                           weights['toxicity'] * toxicity)
        chemical_score = 0.0
        if selected_smiles_column and df.iloc[idx][selected_smiles_column] and not pd.isna(df.iloc[idx][selected_smiles_column]):
            chemical_score = DataStructs.TanimotoSimilarity(mof_fp_rdkit, reference_fp_rdkit)
        total_score = structural_score + weights['chemical'] * chemical_score
        penalty = 0
        penalty_details = {}
        for feat, actual_value in zip(numerical_features, mapped_solution):
            min_val, max_val = ranges[feat]
            if actual_value < min_val:
                deviation = (min_val - actual_value) / (max_val - min_val)
                penalty += 0.2 * deviation ** 2
                penalty_details[feat] = 0.2 * deviation ** 2
                print(f"Penalty for {feat}: actual={actual_value:.4f}, range=[{min_val:.4f}, {max_val:.4f}]")
            elif actual_value > max_val:
                deviation = (actual_value - max_val) / (max_val - min_val)
                penalty += 0.2 * deviation ** 2
                penalty_details[feat] = 0.2 * deviation ** 2
                print(f"Penalty for {feat}: actual={actual_value:.4f}, range=[{min_val:.4f}, {max_val:.4f}]")
            else:
                penalty_details[feat] = 0
        diversity_penalty = 0
        if top_solutions:
            distances = [np.linalg.norm(solution_num - np.array(sol[:len(numerical_features)]))
                         for sol in top_solutions]
            diversity_penalty = 0.03 * (1 - min(distances) / 0.5 if distances else 1)
        results.append((total_score - penalty - diversity_penalty, penalty_details, chemical_score))
    return results

# Lotus Effect Algorithm
class LotusEffectAlgorithm:
    def __init__(self, population_size, max_iterations, df_norm, ranges, df, nbrs, top_k=5):
        self.population_size = population_size
        self.max_iterations = max_iterations
        self.df_norm = df_norm
        self.ranges = ranges
        self.df = df
        self.nbrs = nbrs
        self.top_k = top_k
        self.dimensions = len(numerical_features)
        self.population = np.random.rand(population_size, self.dimensions)
        self.step_size = 0.5 * (df_norm[numerical_features].max() - df_norm[numerical_features].min()).values
        self.top_solutions = []
        self.top_fitness = []
        self.top_mofs = []
        self.fitness_history = []
        self.top_penalties = []
        self.top_chem_scores = []

    def levy_flight(self):
        beta = 1.5
        sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
        u = np.random.normal(0, sigma, size=self.dimensions)
        v = np.random.normal(0, 1, size=self.dimensions)
        return u / np.abs(v)**(1 / beta)

    def optimize(self):
        seen_refcodes = set()
        for iteration in range(self.max_iterations):
            fitness_results = fitness_function_drug_delivery_batch(
                self.population, self.df_norm, self.ranges, self.nbrs, self.df,
                self.top_solutions, reference_fp_rdkit
            )
            fitness_scores = [result[0] for result in fitness_results]
            penalty_details = [result[1] for result in fitness_results]
            chem_scores = [result[2] for result in fitness_results]
            for i, (score, penalties, chem_score) in enumerate(zip(fitness_scores, penalty_details, chem_scores)):
                solution_num = self.population[i]
                solution_combined = np.hstack([solution_num, df_norm.iloc[0]['Fingerprint'], [i]])
                mof_idx = nbrs.kneighbors([solution_combined], n_neighbors=1, return_distance=False)[0][0]
                mof_refcode = self.df.iloc[mof_idx]['Refcode']
                if mof_refcode not in seen_refcodes:
                    if len(self.top_fitness) < self.top_k:
                        self.top_solutions.append(self.population[i].copy())
                        self.top_fitness.append(score)
                        self.top_mofs.append(mof_refcode)
                        self.top_penalties.append(penalties)
                        self.top_chem_scores.append(chem_score)
                        seen_refcodes.add(mof_refcode)
                    else:
                        min_idx = np.argmin(self.top_fitness)
                        if score > self.top_fitness[min_idx]:
                            seen_refcodes.remove(self.top_mofs[min_idx])
                            self.top_solutions[min_idx] = self.population[i].copy()
                            self.top_fitness[min_idx] = score
                            self.top_mofs[min_idx] = mof_refcode
                            self.top_penalties[min_idx] = penalties
                            self.top_chem_scores[min_idx] = chem_score
                            seen_refcodes.add(mof_refcode)
            for i in range(self.population_size):
                self.population[i] += self.step_size * self.levy_flight()
                self.population[i] = np.clip(self.population[i], 0, 1)
            self.fitness_history.append(max(fitness_scores) if fitness_scores else 0)

        sorted_indices = np.argsort(self.top_fitness)[::-1]
        self.top_solutions = [self.top_solutions[i] for i in sorted_indices]
        self.top_fitness = [self.top_fitness[i] for i in sorted_indices]
        self.top_mofs = [self.top_mofs[i] for i in sorted_indices]
        self.top_penalties = [self.top_penalties[i] for i in sorted_indices]
        self.top_chem_scores = [self.top_chem_scores[i] for i in sorted_indices]
        return self.top_solutions, self.top_fitness, self.top_mofs, self.top_penalties, self.top_chem_scores

# Analysis and visualization functions
def analyze_diversity(top_solutions, df_norm, numerical_features):
    kmeans = KMeans(n_clusters=min(4, len(top_solutions)), n_init=10, random_state=42)
    clusters = kmeans.fit_predict(top_solutions)
    distances = np.mean([np.linalg.norm(top_solutions[i] - top_solutions[j])
                         for i in range(len(top_solutions))
                         for j in range(i + 1, len(top_solutions))])
    print(f"Average Euclidean Distance: {distances:.2f}")
    print(f"Number of Clusters: {len(np.unique(clusters))}")
    plt.figure(figsize=(8, 5))
    plt.scatter([s[2] for s in top_solutions], [s[0] for s in top_solutions], c=clusters, cmap='viridis')
    plt.xlabel("PLD (Normalized)")
    plt.ylabel("Void Fraction (Normalized)")
    plt.title("Clustering of Top MOF Solutions")
    plt.savefig('mof_clusters.png')
    plt.close()

def visualize_tradeoffs(top_solutions, top_fitness, df_norm, numerical_features, top_mofs):
    porosity_scores = [sum(df_norm[df_norm['Refcode'] == mof][['void_fraction', 'asa (A^2)', 'pld (A)']].values[0])
                       for mof in top_mofs]
    stability_scores = [sum(df_norm[df_norm['Refcode'] == mof][['max_metal_coordination_n', 'n_sbu_point_of_extension']].values[0])
                        for mof in top_mofs]
    plt.figure(figsize=(8, 5))
    scatter = plt.scatter(porosity_scores, stability_scores, c=top_fitness, cmap='viridis', s=100)
    plt.colorbar(scatter, label='Fitness Score')
    for i, mof in enumerate(top_mofs):
        plt.annotate(mof, (porosity_scores[i], stability_scores[i]))
    plt.xlabel("Porosity Score (Normalized)")
    plt.ylabel("Stability Score (Normalized)")
    plt.title("Trade-offs: Porosity vs Stability for Top MOFs")
    plt.savefig('tradeoffs.png')
    plt.close()

def export_top_mofs(top_mofs, top_fitness, df, df_norm, top_penalties, top_chem_scores):
    results = []
    for mof, fitness, penalties, chem_score in zip(top_mofs, top_fitness, top_penalties, top_chem_scores):
        row = df[df['Refcode'] == mof].iloc[0].to_dict()
        row['Fitness'] = fitness
        row['Chemical_Score'] = chem_score
        row.update({f"Penalty_{k}": v for k, v in penalties.items()})
        results.append(row)
    results_df = pd.DataFrame(results)
    results_df.to_csv('top_mofs_drug_delivery.csv', index=False)
    print("Top MOFs exported to 'top_mofs_drug_delivery.csv'")

def export_mof_smiles_images(df, output_folder='mof_smiles_images', top_n=5):
    os.makedirs(output_folder, exist_ok=True)
    for i, row in df.head(top_n).iterrows():
        refcode = row['Refcode']
        smiles = row.get(selected_smiles_column, '') if selected_smiles_column else ''
        if not smiles or pd.isna(smiles):
            print(f"[SKIP] MOF {refcode} has no valid SMILES.")
            continue
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            file_path = os.path.join(output_folder, f"{refcode}.png")
            Draw.MolToFile(mol, file_path, size=(300, 300))
            print(f"[OK] Saved structure for {refcode} to {file_path}")
        else:
            print(f"[ERROR] Invalid SMILES for {refcode}: {smiles}")

# Run experiments
def run_experiments():
    start_time = time.time()
    lea = LotusEffectAlgorithm(
        population_size=30,
        max_iterations=100,
        df_norm=df_norm,
        ranges=ranges,
        df=df,
        nbrs=nbrs,
        top_k=5
    )
    top_solutions, top_fitness, top_mofs, top_penalties, top_chem_scores = lea.optimize()
    lea_time = time.time() - start_time
    print("LEA Optimization Results:")
    for i, (mof, fitness) in enumerate(zip(top_mofs, top_fitness)):
        print(f"Top {i+1} MOF: {mof}, Fitness: {fitness:.4f}")
    print(f"Optimization Time: {lea_time:.2f}s")
    export_top_mofs(top_mofs, top_fitness, df, df_norm, top_penalties, top_chem_scores)
    print("\nPenalty and Chemical Score Details for Top MOFs:")
    for i, (mof, penalties, chem_score) in enumerate(zip(top_mofs, top_penalties, top_chem_scores)):
        print(f"Top {i+1} MOF: {mof}, Chemical Score: {chem_score:.4f}")
        for feat, penalty in penalties.items():
            if penalty > 0:
                print(f"  {feat}: Penalty={penalty:.4f}")
    analyze_diversity(top_solutions, df_norm, numerical_features)
    visualize_tradeoffs(top_solutions, top_fitness, df_norm, numerical_features, top_mofs)
    export_mof_smiles_images(df[df['Refcode'].isin(top_mofs)])
    plt.figure(figsize=(8, 5))
    plt.plot(lea.fitness_history, label="LEA Optimization")
    plt.xlabel("Iterations")
    plt.ylabel("Fitness Score (Higher = Better)")
    plt.title("MOF Optimization for Drug Delivery - Convergence Curve")
    plt.legend()
    plt.savefig('convergence_curve.png')
    plt.close()
    plt.figure(figsize=(8, 5))
    plt.hist(df['pld (A)'], bins=30, alpha=0.7, label="All MOFs")
    for mof in top_mofs:
        plt.axvline(df[df['Refcode'] == mof]['pld (A)'].values[0],
                    color='red', linestyle='dashed', alpha=0.5)
    plt.axvline(df[df['Refcode'] == top_mofs[0]]['pld (A)'].values[0],
                color='red', linestyle='dashed', label="Top MOFs")
    plt.xlabel("Pore Limiting Diameter (PLD, Å)")
    plt.ylabel("Frequency")
    plt.title("PLD Distribution in MOF Dataset")
    plt.legend()
    plt.savefig('pld_distribution.png')
    plt.close()

if __name__ == "__main__":
    run_experiments()


Using SMILES column: ligand_smile
Non-null SMILES count: 13727
Sample SMILES: ['[O]P(=O)(CN1CCN(CC1)CP(=O)([O])[O])[O]', 'C(=O)(c1cnc(cc1)C(=O)[O])[O]', 'n1ccc(cc1)c1ccc(cc1)c1ccc(cc1)C(=O)[O]', 'N1=C(N=C(c2ccc(C)cc2)[N]1)c1ccc(cc1)C', '[O]C(=O)CC(CC(=O)[O])CC(=O)[O]']
Found 589 invalid SMILES:
Invalid SMILES for Refcode FIHBUV: O=C1[NH](C=CC=C1C(=O)[O])Cc1ccc(C(=O)[O])cc1
Invalid SMILES for Refcode CARKEO: n1ccc(cc1)C1=[N]=C(N=C([N]1)c1ccncc1)c1ccncc1
Invalid SMILES for Refcode WIZQUT: N1=C2/C(=C\3/C=C/C(=C(\c4cc(OC)c(c(OC)c4)OC)/C4=N/C(=C(/c5cc(OC)c(OC)c(OC)c5)\C5=CC=C(/C(=C\1/C=C2)/c1ccc(OCc2cn(CC[NH]6C=C[C](C=C6)[C]6C=C[NH](C=C6)C)nn2)cc1)[N]5)/C=C4)/[N]3)/c1cc(c(c(c1)OC)OC)OC
Invalid SMILES for Refcode HAYYOY: [N]1c2cc3c(cc2[N][NH2]1)[N][NH2][N]3
Invalid SMILES for Refcode REXFOR: n1ccc(C2=[N]=C(c3ccncc3)N=C(c3ccncc3)[N]2)cc1
Invalid SMILES for Refcode QODHUO09: n1ccc(C2=[N]=C(c3ccncc3)N=C(c3ccncc3)[N]2)cc1
Invalid SMILES for Refcode PEGBEK: N1=C([N]C(=N1)CC1=[N]=C(N=N1)C)C
Invali

Below is a table summarizing the key characteristics and suitability of the top five Metal-Organic Frameworks (MOFs) from the provided `top_mofs_drug_delivery.csv` file, optimized for drug delivery with ibuprofen as the reference molecule. The table includes structural properties, chemical scores, toxicity considerations, stability/synthesis factors, fitness scores, penalties, and recommended applications, addressing their suitability for drug delivery.

| **Refcode** | **Void Fraction** | **ASA (Å²)** | **PLD (Å)** | **Max Metal Coordination** | **SBU Points** | **Metal (Toxicity Score)** | **Ligand SMILES** | **Chemical Score** | **Fitness Score** | **Penalties** | **Stability/Synthesis** | **Suitability for Drug Delivery** | **Recommended Application** |
|-------------|-------------------|--------------|-------------|---------------------------|----------------|----------------------------|-------------------|--------------------|-------------------|---------------|-------------------------|-----------------------------------|-----------------------------|
| **JARSED**  | 0.6174            | 925.213      | 14.78637    | 4                         | 4              | Zn (0.2, Low)              | `c1c(C(=O)[O])cc(C(=O)[O])cc1C(=O)[O]` (Tricarboxylate) | 0.1563             | 0.3073            | Void: 0.0260, ASA: 0.0009, PLD: 0.0038 | High (Simple Zn node, common linker) | **High**: Large pores, high porosity, biocompatible Zn, stable framework. Moderate chemical compatibility. | Oral/topical delivery of ibuprofen or similar drugs (e.g., aspirin). |
| **MIJFAM**  | 0.5558            | 1073.42      | 15.49461    | 6                         | 4              | Cu (0.3, Moderate)         | `n1ccc(CCCc2ccncc2)cc1` (Dipyridyl) | 0.1389             | 0.2971            | Void: 0.0101, ASA: 0.0114, PLD: 0.0075 | High (Stable Cu node, standard linker) | **High**: Highest ASA and PLD, ideal for high drug loading and diffusion. Cu toxicity needs validation. | Controlled release in injectable/implantable systems for hydrophobic drugs. |
| **XIFHOL**  | 0.4686            | 696.057      | 12.18665    | 9                         | 2              | Mo (0.5, High)             | `n1ccc(cc1)CCCc1ccncc1` (Dipyridyl) | 0.1389             | 0.2910            | Void: 0.0002 | Moderate (Complex Mo node, lower porosity) | **Low**: Suitable for non-biomedical applications due to Mo toxicity and lower porosity. | Experimental delivery in non-biological contexts (e.g., agricultural). |
| **XOXHUQ**  | 0.5886            | 420.068      | 11.01939    | 11                        | 4              | U (0.5, Very High)         | `O=C([O])Cc1cccc(c1)CC(=O)[O]` (Dicarboxylate) | 0.2222             | 0.2815            | Void: 0.0177, Coord: 0.0031 | Low (Uranium node, synthetic challenges) | **Very Low**: Unsuitable for biomedical use due to uranium toxicity, despite high chemical compatibility. | Non-biological applications (e.g., catalytic drug synthesis). |
| **JAQTON**  | 0.6294            | 751.336      | 14.96315    | 13                        | 6              | La (0.5, High)             | `[O]C(=O)c1cc(C(=O)[O])ccc1` (Dicarboxylate) | 0.1714             | 0.2813            | Void: 0.0300, PLD: 0.0046, Coord: 0.0281 | Low (Complex La node, high penalty) | **Moderate**: Suitable for research with large drugs, pending La toxicity validation. | Research delivery of larger drug molecules. |

### Table Notes
- **Void Fraction**: Higher values (e.g., JAQTON: 0.6294) indicate greater pore volume for drug storage.
- **ASA (Accessible Surface Area)**: Higher values (e.g., MIJFAM: 1073.42 Å²) enable more drug-surface interactions.
- **PLD (Pore Limiting Diameter)**: Values >10–12 Å (ibuprofen’s size) ensure drug accessibility; MIJFAM (15.49461 Å) is largest.
- **Max Metal Coordination**: Lower numbers (e.g., JARSED: 4) suggest simpler, stable nodes; high numbers (e.g., JAQTON: 13) may reduce stability.
- **SBU Points**: Moderate values (2–6) balance framework robustness and synthesis feasibility.
- **Metal (Toxicity Score)**: Zn (0.2) and Cu (0.3) are safer; Mo, U, La (0.5) raise biocompatibility concerns, especially U (radioactive).
- **Ligand SMILES**: Carboxylate-based linkers (JARSED, XOXHUQ, JAQTON) align better with ibuprofen’s chemistry than dipyridyl (MIJFAM, XIFHOL).
- **Chemical Score**: Tanimoto similarity to ibuprofen (0.1389–0.2222); higher scores (e.g., XOXHUQ: 0.2222) indicate better chemical compatibility.
- **Fitness Score**: Balances structural properties (65%) and chemical similarity (35%), adjusted for penalties and toxicity.
- **Penalties**: Reflect deviations from ideal ranges; JAQTON’s high coordination penalty (0.0281) suggests instability.
- **Stability/Synthesis**: Assessed by metal node simplicity, linker commonality, and penalties.
- **Suitability**: Evaluates overall fit for drug delivery based on porosity, pore size, biocompatibility, and chemical compatibility.
- **Recommended Application**: Specific drug delivery contexts where each MOF excels, considering limitations.

### Key Insights
- **Best for Drug Delivery**: **JARSED** (Zn, high porosity, stable, biocompatible) and **MIJFAM** (Cu, highest ASA/PLD, moderate toxicity).
- **Moderate Suitability**: **JAQTON** (La, high porosity but complex node and toxicity concerns).
- **Low Suitability**: **XIFHOL** (Mo, toxicity and low porosity) and **XOXHUQ** (U, severe toxicity despite high chemical score).
- **Improvements Needed**: Filter out high-toxicity metals (Mo, U, La), test alternative SMILES columns (e.g., `linker_smile`), and relax penalty ranges to boost fitness scores.

This table provides a clear comparison of the MOFs’ strengths and weaknesses for drug delivery, guiding their practical use and further optimization. If you need further analysis or modifications (e.g., rerun with updated parameters), please share additional outputs or a sample of `MOF.csv`!